# Temporary workspace

For porting ISCHIA co-occurence function to Julia

In [50]:
using Muon
using RData
using Rmath
using Random
using DataFrames
using Statistics
using ProgressMeter
using Combinatorics

In [56]:
mdata = readh5mu("../data/mudata.h5mu")
lr_network = load("../data/lr_network.rds")

gene_names = mdata["SCT"].var.name
mdata["Spatial"].var_names = mdata["Spatial"].var.name

# Create LR_Pairs column
lr_network[!, :LR_Pairs] = string.(lr_network.from, "_", lr_network.to);
lr_network = lr_network[:, [:from, :to, :LR_Pairs]]

# Filter lr_network based on conditions
from_filter = in.(lr_network[:, :from], Ref(gene_names))
to_filter = in.(lr_network[:, :to], Ref(gene_names))
all_LR_network = lr_network[from_filter .& to_filter, :]

# To reduce the computation time for this example, we randomly sample from the whole dataset of LR interactions

# all_LR_network = all_LR_network[shuffle(1:size(all_LR_network_exp, 1)), :]
all_LR_network = all_LR_network[1501:min(2000, end), :]

# Extract unique genes and common genes
all_LR_genes = unique(vcat(all_LR_network[:, :from], all_LR_network[:, :to]))
all_LR_genes_comm = intersect(all_LR_genes, collect(gene_names));

# Create LR.pairs and LR.pairs.AllCombos
LR_pairs = all_LR_network[:, :LR_Pairs]
all_combos = [join(combo, "_") for combo in combinations(all_LR_genes_comm, 2)];

adata = mdata["Spatial"]
COI = ["CC4"]
Condition = unique(adata.obs[!, "orig.ident"])
LR_list = all_LR_genes_comm
LR_pairs = LR_pairs
exp_th = 1
corr_th = 0.2

println("Preparing L-R presence/absence matrix")

# Subset the expression matrix for the interested ligands and receptors
spatial_obj_exp_LR_subset_raw = adata[:, in.(adata.var.name, Ref(LR_list))]

# Binarize the expression matrix based on the expression threshold
spatial_obj_exp_LR_subset_raw_binary = spatial_obj_exp_LR_subset_raw.layers["counts"] .> exp_th
spatial_obj_exp_LR_subset_raw.layers["binary"] = spatial_obj_exp_LR_subset_raw_binary

LR_subset_raw_binary_mask_col = vec(sum(spatial_obj_exp_LR_subset_raw_binary, dims=1) .> 0)
LR_subset_raw_binary_mask_row = vec(sum(spatial_obj_exp_LR_subset_raw_binary, dims=2) .> 0)

LR_presence_absence = spatial_obj_exp_LR_subset_raw[LR_subset_raw_binary_mask_row, LR_subset_raw_binary_mask_col]


# Filter spots based on COI and Condition
mask = (adata.obs[:, "CompositionCluster_CC"] .∈ Ref(COI)) .& (adata.obs[:, "orig.ident"] .∈ Ref(Condition))
COI_spots = adata.obs_names[mask]
rest_of_spots = setdiff(adata.obs_names, COI_spots)

println("Calculating L-R pairs correlation")
COI_cors_adata = spatial_obj_exp_LR_subset_raw[mask, :]
COI_cors = cor(Array(COI_cors_adata.layers["counts"]))
COI_cors[isnan.(COI_cors)] .= 0.0;


Preparing L-R presence/absence matrix
Calculating L-R pairs correlation


In [57]:
LR_list

243-element Vector{String}:
 "CSF1"
 "WNT11"
 "PTN"
 "WNT3"
 "IHH"
 "SHH"
 "CCL26"
 "DHH"
 "ADAM17"
 "MDK"
 ⋮
 "GFRA1"
 "DCC"
 "ITGA1"
 "ROBO3"
 "ITGB4"
 "ITGA6"
 "UNC5A"
 "ROBO2"
 "TYRO3"

In [59]:
sum(adata.var_names .== "CSF1")

1

In [61]:
adata[:, LR_list].var.name

243-element Vector{String}:
 "CSF1"
 "WNT11"
 "PTN"
 "WNT3"
 "IHH"
 "SHH"
 "CCL26"
 "DHH"
 "ADAM17"
 "MDK"
 ⋮
 "GFRA1"
 "DCC"
 "ITGA1"
 "ROBO3"
 "ITGB4"
 "ITGA6"
 "UNC5A"
 "ROBO2"
 "TYRO3"

In [29]:
adata[LR_list, :].var.name

KeyError: KeyError: key "CSF1" not found

In [ ]:

println("Preparing for cooccurrence")
common_spots = intersect(LR_presence_absence.obs_names, COI_spots)
coocur_COI = LR_presence_absence[common_spots, :]
coocur_COI_exp = DataFrame(Matrix(transpose(coocur_COI.layers["binary"])), common_spots)

describe(coocur_COI_exp)

## Co-Occurence Part

The code above is used to setup the part below. Now both `R` and `Julia` code has same variables and data. So it'll be easier/possible to compare our output with theirs.

In [6]:
mat = coocur_COI_exp
row_names = coocur_COI.var.name
type = "spp_site"
thresh = true
spp_names = true
true_rand_classifier = 0.1
prob = "hyper"
site_mask = nothing
only_effects = false
eff_standard = true
eff_matrix = false;

In [7]:
if type == "spp_site"
    spp_site_mat = mat
elseif type == "site_spp"
    spp_site_mat = transpose(mat)
else
    error("Invalid 'type' parameter")
end


# spp_site_mat
;

In [10]:
if spp_names
    spp_key = DataFrame(num=1:nrow(spp_site_mat), spp=row_names)
end
spp_key

Row,num,spp
,Int64,String
1,1,AGRN
2,2,TNFRSF14
3,3,TNFRSF1B
4,4,EPHB2
5,5,PTCH2
6,6,LRP8
7,7,TGFBR3
8,8,SORT1
9,9,CSF1


In [304]:
"""
Calculate the co-occurrence matrix N from a binary species-site matrix.

This function creates a species by species matrix of potential co-occurring sites (N) from a binary species by site matrix, where 1 represents potential occupancy, and 0 indicates species absence.

# Arguments
- `mat::Matrix{Int}`: A binary species by site matrix.

# Returns
A species by species matrix where the upper triangle contains N for each species pair.

# Examples
```julia
# Define a binary species by site matrix
# species_matrix = rand(Bool, num_species, num_sites)

# Calculate the co-occurrence matrix N
# cooccurrence_matrix = create_N_matrix(species_matrix)

"""
function calculate_cooccurrence_matrix(mat::Matrix{Int})
    num_species = size(mat, 1)
    cooccurrence_matrix = zeros(Int, num_species, num_species)
    
    for i in 1:num_species
        for j in (i + 1):num_species
            cooccurrence_matrix[i, j] = sum(mat[i, :] .* mat[j, :])
            cooccurrence_matrix[j, i] = cooccurrence_matrix[i, j]
        end
    end
    
    return cooccurrence_matrix    
end

# finches = load("../data/finches.rda")["finches"]
# N_matrix = rand(Bool, nrow(finches), ncol(finches))
# create_N_matrix(N_matrix)


calculate_cooccurrence_matrix

In [305]:
if !isnothing(site_mask)
    if size(site_mask) == size(spp_site_mat)
        N_matrix = calculate_cooccurrence_matrix(site_mask)
    else
        error("Incorrect dimensions for 'site_mask', aborting.")
    end
else
    site_mask = ones(Int, size(spp_site_mat))
    N_matrix = size(spp_site_mat, 2) * ones(Int, (size(spp_site_mat, 1), size(spp_site_mat, 1)))
end
;

In [306]:
# spp_site_mat[spp_site_mat .> 0] .= 1

In [307]:
tsites = size(spp_site_mat, 2)
nspp = size(spp_site_mat, 1)
spp_pairs = binomial(nspp, 2);

In [308]:
spp_pairs

15576

In [309]:
obs_cooccur = zeros(Int, spp_pairs, 3)

for x = [:prob_cooccur,:exp_cooccur]
    @eval $x = zeros(Real, spp_pairs, 3)
end

incidence = zeros(Int, size(N_matrix));
prob_occur = zeros(size(N_matrix));

In [310]:
size(N_matrix)

(177, 177)

In [314]:
size(zeros(Int, 177, 177))

(177, 177)

In [247]:
mat_matrix = Matrix(mat)
for spp in ProgressBar(1:nspp)
    if spp < nspp
        for spp_next in (spp + 1):nspp
            incidence[spp, spp_next] = sum(site_mask[spp, :] .* site_mask[spp_next, :] .* mat_matrix[spp, :])
            incidence[spp_next, spp] = sum(site_mask[spp, :] .* site_mask[spp_next, :] .* mat_matrix[spp_next, :])
        end
    end
end

0.0%┣                                              ┫ 0/177 [00:00<00:00, -0s/it]
70.1%┣████████████████████████████▊            ┫ 124/177 [00:00<00:00, 2.4kit/s]
100.0%┣████████████████████████████████████████┫ 177/177 [00:00<00:00, 3.1kit/s]
100.0%┣████████████████████████████████████████┫ 177/177 [00:00<00:00, 3.1kit/s]


In [248]:
prob_occur .= incidence ./ N_matrix;
describe(DataFrame(prob_occur, :auto));

In [317]:
size(mat)

(177, 13)

In [249]:
row = 0
for spp in ProgressBar(1:nspp)
    if spp < nspp
        for spp_next in (spp + 1):nspp
            pairs = sum(mat_matrix[spp, site_mask[spp, :] .* site_mask[spp_next, :] .== 1] .== 1 .&
                mat_matrix[spp_next, site_mask[spp, :] .* site_mask[spp_next, :] .== 1] .== 1)
            row += 1
            obs_cooccur[row, 1] = spp
            obs_cooccur[row, 2] = spp_next
            obs_cooccur[row, 3] = pairs
            prob_cooccur[row, 1] = spp
            prob_cooccur[row, 2] = spp_next
            prob_cooccur[row, 3] = prob_occur[spp, spp_next] * prob_occur[spp_next, spp]
            exp_cooccur[row, 1] = spp
            exp_cooccur[row, 2] = spp_next
            exp_cooccur[row, 3] = prob_cooccur[row, 3] * N_matrix[spp, spp_next]
        end
    end
end

0.0%┣                                              ┫ 0/177 [00:00<00:00, -0s/it]


42.4%┣█████████████████▉                        ┫ 75/177 [00:00<00:00, 1.5kit/s]
100.0%┣████████████████████████████████████████┫ 177/177 [00:00<00:00, 2.3kit/s]
100.0%┣████████████████████████████████████████┫ 177/177 [00:00<00:00, 2.2kit/s]


In [250]:
# describe(DataFrame(obs_cooccur, :auto))
# describe(DataFrame(prob_cooccur, :auto))
# describe(DataFrame(exp_cooccur, :auto))
# Verified with R code

In [251]:
if thresh
    n_pairs = size(prob_cooccur, 1)
    mask = exp_cooccur[:, 3] .>= 1
    prob_cooccur = prob_cooccur[mask, :]
    obs_cooccur = obs_cooccur[mask, :]
    exp_cooccur = exp_cooccur[mask, :]
    n_omitted = n_pairs - size(prob_cooccur, 1)
end

15540

In [252]:
output = DataFrame(sp1=Integer[], sp2=Integer[], sp1_inc=Integer[], sp2_inc=Integer[],
                       obs_cooccur=Integer[], prob_cooccur=Real[], exp_cooccur=Real[],
                       p_lt=Real[], p_gt=Real[])

Row,sp1,sp2,sp1_inc,sp2_inc,obs_cooccur,prob_cooccur,exp_cooccur,p_lt,p_gt
,Integer,Integer,Integer,Integer,Integer,Real,Real,Real,Real


In [253]:
"""
    calculate_conditional_probability(max_successes, successes, min_successes, total_trials)

Calculate the conditional probability using binomial coefficients.

# Arguments
- `successes::Int`: The number of successful trials.
- `min_successes::Int`: The minimum number of successful trials.
- `max_successes::Int`: The maximum number of successful trials.
- `total_trials::Int`: The total number of trials.

# Returns
The calculated conditional probability as a floating-point number.
"""
function calculate_conditional_probability(
    successes::Int, min_successes::Int, max_successes::Int, total_trials::Int
    )::Real

    # Calculate the numerator using binomial coefficients
    numerator = binomial(max_successes, successes) * binomial(total_trials - max_successes, min_successes - successes)
    
    # Calculate the denominator using binomial coefficients
    denominator = binomial(total_trials, min_successes)
    
    # Calculate and return the conditional probability
    return numerator / denominator
end

# calculate_conditional_probability(4, 10, 20, 50)
# Verified

calculate_conditional_probability

In [254]:
maximum(incidence)

8

In [255]:
# prob = "comb"
for row in ProgressBar(1:size(obs_cooccur, 1))
    sp1 = obs_cooccur[row, 1]
    sp2 = obs_cooccur[row, 2]
    sp1_inc = convert(Integer, incidence[sp1, sp2])
    sp2_inc = convert(Integer, incidence[sp2, sp1])
    max_inc = max(sp1_inc, sp2_inc)
    min_inc = min(sp1_inc, sp2_inc)
    nsite = N_matrix[sp1, sp2]
    psite = nsite + 1
    prob_share_site = zeros(Float64, psite)
    
    if prob == "hyper"
        if !only_effects
            all_probs = phyper.(0:min_inc, min_inc, nsite - min_inc, max_inc)
            prob_share_site[1] = all_probs[1]
            for j in 2:length(all_probs)
                prob_share_site[j] = all_probs[j] - all_probs[j - 1]
            end
        else
            for j in 0:nsite
                if (sp1_inc + sp2_inc) <= (nsite + j)
                    if j <= min_inc
                        prob_share_site[j + 1] = 1
                    end
                end
            end
        end
    end

    if prob == "comb"
        if !only_effects
            for j in 0:nsite
                if (sp1_inc + sp2_inc) <= (nsite + j)
                    if j <= min_inc
                        prob_share_site[j + 1] = calculate_conditional_probability(j, min_inc, max_inc, nsite)
                    end
                end
            end
        else
            for j in 0:nsite
                if (sp1_inc + sp2_inc) <= (nsite + j)
                    if j <= min_inc
                        prob_share_site[j + 1] = 1
                    end
                end
            end
        end
    end

    p_lt = 0.0
    p_gt = 0.0
    for j in 0:nsite
        if j <= obs_cooccur[row, 3]
            p_lt += prob_share_site[j + 1]
        end
        if j >= obs_cooccur[row, 3]
            p_gt += prob_share_site[j + 1]
        end
        if j == obs_cooccur[row, 3]
            p_exactly_obs = prob_share_site[j + 1]
        end
    end
    
    p_lt = round(p_lt, digits=5)
    p_gt = round(p_gt, digits=5)
    p_exactly_obs = round(p_exactly_obs, digits=5)
    prob_cooccur[row, 3] = round(prob_cooccur[row, 3], digits=3)
    exp_cooccur[row, 3] = round(exp_cooccur[row, 3], digits=1)
    
    push!(output, [sp1, sp2, sp1_inc, sp2_inc, obs_cooccur[row, 3],
                   prob_cooccur[row, 3], exp_cooccur[row, 3], p_lt, p_gt])
end

# describe(output)
# verified by R

0.0%┣                                               ┫ 0/36 [00:00<00:00, -0s/it]


100.0%┣███████████████████████████████████████████┫ 36/36 [00:00<00:00, 840it/s]
100.0%┣███████████████████████████████████████████┫ 36/36 [00:00<00:00, 828it/s]


In [281]:
if spp_names
    sp1_name = leftjoin(DataFrame(order = 1:length(output.sp1), sp1 = output.sp1), spp_key, on = :sp1 => :num, makeunique = true)
    sp2_name = leftjoin(DataFrame(order = 1:length(output.sp2), sp2 = output.sp2), spp_key, on = :sp2 => :num, makeunique = true)
    
    output.sp1_name = sp1_name[sortperm(sp1_name.order), "spp"]
    output.sp2_name = sp2_name[sortperm(sp2_name.order), "spp"]
end

output; #verified

In [287]:
true_rand = count(x -> (x.p_gt >= 0.05 && x.p_lt >= 0.05 && abs(x.obs_cooccur - x.exp_cooccur) <= (tsites * true_rand_classifier)), eachrow(output))

20

In [288]:
output_dict = Dict("results"=>output,
                       "positive"=>count(x -> x.p_gt < 0.05, eachrow(output)),
                       "negative"=>count(x -> x.p_lt < 0.05, eachrow(output)),
                       "co_occurrences"=>count(x -> x.p_gt < 0.05 || x.p_lt < 0.05, eachrow(output)),
                       "pairs"=>size(output, 1),
                       "random"=>true_rand,
                       "unclassifiable"=>size(output, 1) - (true_rand + count(x -> x.p_gt < 0.05, eachrow(output)) + count(x -> x.p_lt < 0.05, eachrow(output))),
                       "sites"=>N_matrix,
                       "species"=>nspp,
                       "percent_sig"=>count(x -> x.p_gt < 0.05 || x.p_lt < 0.05, eachrow(output)) / size(output, 1) * 100,
                       "true_rand_classifier"=>true_rand_classifier)

Dict{String, Any} with 11 entries:
  "co_occurrences"       => 9
  "unclassifiable"       => 7
  "percent_sig"          => 25.0
  "results"              => 36×11 DataFrame…
  "species"              => 177
  "random"               => 20
  "sites"                => [13 13 … 13 13; 13 13 … 13 13; … ; 13 13 … 13 13; 1…
  "negative"             => 0
  "pairs"                => 36
  "positive"             => 9
  "true_rand_classifier" => 0.1

In [292]:
if spp_names
    output_dict["spp_key"] = spp_key
    output_dict["spp_names"] = row_names
else
    output_dict["spp_names"] = 1:size(spp_site_mat, 1)
end

177-element Vector{String}:
 "AGRN"
 "TNFRSF14"
 "TNFRSF1B"
 "EPHB2"
 "PTCH2"
 "LRP8"
 "TGFBR3"
 "SORT1"
 "CSF1"
 "NOTCH2"
 ⋮
 "THBD"
 "SDC4"
 "MMP9"
 "EDN3"
 "APP"
 "IFNAR2"
 "CSF2RA"
 "IL3RA"
 "CXCR3"

In [294]:
if thresh
    output_dict["omitted"] = n_omitted
    output_dict["pot_pairs"] = n_pairs
end

# output_dict["class"] = "cooccur"

15576

In [ ]:

function ISCHIA_cooccur(mat, type="spp_site", thresh=true, spp_names=false,
                         true_rand_classifier=0.1, prob="hyper", site_mask=nothing,
                         only_effects=false, eff_standard=true, eff_matrix=false)

    
    
    
    
    
    
    
    
    return output_dict
    # if !only_effects
    #     return output_dict
    # else
    #     return effect_sizes(output_dict, standardized=eff_standard, matrix=eff_matrix)
    # end
end
